In [18]:
from randVideo import VideoRenderer, VideoObject
import matplotlib.pyplot as plt
import os
import json
from torchvision import transforms
from preprocess_utensils import (
    get_gray_image,
    get_boundary_iou,
    get_boundary_points,
    uniform_sample_points,
)
import numpy as np
import cv2
import random
import torch
from PIL import Image

In [6]:
class OneHundredVideoGenerator(VideoRenderer):
    def __init__(self, root="100Videos") -> None:
        super().__init__(root)
    
    def render_one_video(
        self,
        video_object: VideoObject,
        index: int,
        occulusion: bool = False,
    ):
        if occulusion:
            video_name = video_object.name + "_occlusion"
            self.occlusion_object.x = video_object.x
            self.occlusion_object.y = video_object.y
        else:
            video_name = video_object.name
        video_name = f"{video_name}_{index}"
        path = f"./tmp_videos/{self.root}/{video_name}"
        self.video_path_dict[video_name] = []

        os.makedirs(path, exist_ok=True)
        for i in range(20):
            plt.figure()
            ax = plt.gca()
            plt.style.use("dark_background")
            video_object.random_move()
            video_object.draw(ax)
            if occulusion:
                self.occlusion_object.random_move()
                self.occlusion_object.draw(ax)
            plt.xlim(-10, 10)
            plt.ylim(-10, 10)
            plt.axis("off")
            save_path = f"{path}/{i:02d}.png"
            self.video_path_dict[video_name].append(save_path)
            plt.savefig(save_path)
            plt.close()
    
    def render_all_video(self):
        for i in range(10):
            for video_object in self.objects:
                self.render_one_video(video_object, i, occulusion=False)
                self.render_one_video(video_object, i, occulusion=True)

generator = OneHundredVideoGenerator()
generator.render_all_video()

In [8]:
generator.video_path_dict.keys()

dict_keys(['ellipse_0', 'ellipse_occlusion_0', 'triangle_0', 'triangle_occlusion_0', 'rectangle_0', 'rectangle_occlusion_0', 'arrow_0', 'arrow_occlusion_0', 'fancybbox_0', 'fancybbox_occlusion_0', 'ellipse_1', 'ellipse_occlusion_1', 'triangle_1', 'triangle_occlusion_1', 'rectangle_1', 'rectangle_occlusion_1', 'arrow_1', 'arrow_occlusion_1', 'fancybbox_1', 'fancybbox_occlusion_1', 'ellipse_2', 'ellipse_occlusion_2', 'triangle_2', 'triangle_occlusion_2', 'rectangle_2', 'rectangle_occlusion_2', 'arrow_2', 'arrow_occlusion_2', 'fancybbox_2', 'fancybbox_occlusion_2', 'ellipse_3', 'ellipse_occlusion_3', 'triangle_3', 'triangle_occlusion_3', 'rectangle_3', 'rectangle_occlusion_3', 'arrow_3', 'arrow_occlusion_3', 'fancybbox_3', 'fancybbox_occlusion_3', 'ellipse_4', 'ellipse_occlusion_4', 'triangle_4', 'triangle_occlusion_4', 'rectangle_4', 'rectangle_occlusion_4', 'arrow_4', 'arrow_occlusion_4', 'fancybbox_4', 'fancybbox_occlusion_4', 'ellipse_5', 'ellipse_occlusion_5', 'triangle_5', 'triangle

In [10]:
with open("./tmp_videos/100Videos/video_path_dict.json", "w") as f:
    json.dump(generator.video_path_dict, f)

In [19]:
class OneHundredRawset:
    def __init__(self) -> None:
        with open("./tmp_videos/100Videos/video_path_dict.json", "r") as f:
            self.video_path_dict = json.load(f)
        self.video_names = list(self.video_path_dict.keys())
        self.video_names.sort()
        self.data_set = []
        self.transform = transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )
        for video_name in self.video_names:
            self.data_set.append([])
            for frame_path in self.video_path_dict[video_name]:
                frame = Image.open(frame_path).convert("RGB")
                frame = self.transform(frame)
                sgm = get_gray_image(frame_path)
                boundary_points = get_boundary_points(sgm)
                if boundary_points is None:
                    boundary = torch.zeros((80, 2))
                else:
                    boundary = uniform_sample_points(boundary_points, 80)
                    boundary = torch.tensor(boundary).int()
                sgm[sgm > 0] = 1
                sgm = torch.Tensor(sgm)
                self.data_set[-1].append((frame, boundary, sgm))

    def get_item(self, video_idx, frame_idx):
        return self.data_set[video_idx][frame_idx]

one_hundred_rawset = OneHundredRawset()

In [20]:
from tenLoader import TenDataset

In [21]:
one_hundred_dataset = TenDataset(one_hundred_rawset)